## Overview

Hello! Welcome to the architecture setup notebook, where we will be installing all requirements and outline the basic architecture of our AlexNet model (whose performance will be compared to our custom model, EfficentNet, and ConvNeXt). 


The cell below handles our initial requirements installation:

In [1]:
!pip3 install -r ../../requirements.txt

## Data Preprocessing

As part of our data preprocessing, we will split the down-scaled lung dataset from the original dataset into a train/test split. 

Note that we will be using five-fold cross-validation for testing later, hence we will not be partioning an additional validation set. 

After splitting our data, we will then feed the training set into our models. Here, we will specifically feed it into the AlexNet model. 

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torch.utils.data import SubsetRandomSampler
from torchvision.datasets import ImageFolder
from sklearn.model_selection import KFold

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import os

from sklearn.decomposition import PCA

The code below extracts images from our dataset, resizes each into a fourth their original size (768 -> 192), and converts them into Torch tensors. The ImageFolder class allows us to lazyload our images to preserve our computational power.

In [11]:
# Check current device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to our lung_image_sets
data_dir = "../../lung_colon_image_set/lung_image_sets"

# Define resized size of images (Put this back to 192 later, recommended size of 224)
resized_size = 224

# Convert images into Tensors
tensor_data = transforms.Compose([
  transforms.Resize((resized_size, resized_size)),   # Cut image into a fourth of original size
  transforms.ToTensor()
])

# Load the dataset using ImageFolder
data = ImageFolder(root=data_dir, transform=tensor_data)

# Split the dataset into train and test sets
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train, test = torch.utils.data.random_split(data, [train_size, test_size])

# Create KFold object with 5 folds
num_folds = 5
kfold = KFold(n_splits=5, shuffle=True, random_state=231)

## Model Initialization
We will initialize the AlexNet model using Pytorch's Torchvision pretrained ConvNeXt model and remove the final layer to perform feature extraction on our data.

In [12]:
from torchvision.models import convnext_base

# Define the ConvNeXtCNN model
class ConvNeXtCNN(nn.Module):
    def __init__(self, device):
        super(ConvNeXtCNN, self).__init__()
        
        # Load the pre-trained ConvNeXt model as our feature extractor
        self.convnext = convnext_base(pretrained=True)
        
        # Replace the last layer with an Identity matrix (essentially removes the last FC layer)
        self.convnext.classifier = nn.Identity()

        # Freeze our ConvNeXt model
        self.convnext.requires_grad_(False)
        
        # Convert to device
        self.to(device=device)

    def forward(self, x):
        # Pass the input through the ConvNeXt model
        x = self.convnext(x)
        return x

# Define instance of our ConvNeXt model
model = ConvNeXtCNN(device=device)

### ConvNeXt + SVM
For our first situation, we will use SVM to do classification on our extracted features.

In [13]:
# Define our SVM classifier
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

# Construct results dict to track training
results = {}

# K-Fold Cross Validation
for fold, (train_indices, val_indices) in enumerate(kfold.split(train), 1):
    print(f'Fold {fold}')

    # Create data samplers for train and validation sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Create data loaders for train and validation sets
    train_loader = DataLoader(train, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(train, batch_size=32, sampler=val_sampler)
    
    # Extract features and labels for the training set using ConvNeXt
    train_features = []
    train_labels = []
    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            train_features.append(outputs.cpu().numpy())
            train_labels.append(labels.cpu().numpy())
    train_features = np.concatenate(train_features)
    train_labels = np.concatenate(train_labels)

    # Train the SVM classifier
    svm_classifier.fit(train_features, train_labels)
    
    # Extract features and labels for the validation set using ConvNeXt
    val_features = []
    val_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            val_features.append(outputs.cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    val_features = np.concatenate(val_features)
    val_labels = np.concatenate(val_labels)

    # Evaluate the classifier on the validation set and extract metrics
    val_predictions = svm_classifier.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)

    results[fold] = accuracy
    print(f'Fold {fold} Accuracy: {accuracy:.4f}')

# Print the average metrics across all folds
average_accuracy = np.mean(list(results.values()))

print(f'\nK-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
print('--------------------------------')
for fold in results:
    print(f'Fold {fold}: {results[fold]:.4f}')
print(f'Average: {average_accuracy:.4f}')

Fold 1


KeyboardInterrupt: 

## ConvNeXt + Softmax Classifier Training and Testing
We will perform k-fold cross-validation testing on the Softmax classifier, which is trained the on features extracted by our ConvNeXt model.

In [2]:
# Construct results dict to track training
results = {}

# Create the softmax classifier pipeline via a Logistic Regression with Softmax activation.
softmax_classifier = make_pipeline(
    StandardScaler(),
    LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=400, C=1.0, random_state=231)
)

# K-Fold Cross Validation
for fold, (train_indices, val_indices) in enumerate(kfold.split(train), 1):
    print(f'Fold {fold}')

    # Create data samplers for train and validation sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Create data loaders for train and validation sets
    train_loader = DataLoader(train, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(train, batch_size=32, sampler=val_sampler)

    # Extract features and labels for the training set using ConvNeXt
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    features = np.concatenate(features)
    labels = np.concatenate(labels)

    # Train the softmax classifier
    softmax_classifier.fit(features, labels)

    # Extract features and labels for the validation set using ConvNeXt
    val_features = []
    val_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            val_features.append(outputs.cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    val_features = np.concatenate(val_features)
    val_labels = np.concatenate(val_labels)

    # Evaluate the classifier on the validation set and extract metrics
    val_predictions = softmax_classifier.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)

    results[fold] = accuracy
    print(f'Fold {fold} Accuracy: {accuracy:.4f}')

# Print the average metrics across all folds
average_accuracy = np.mean(list(results.values()))

print(f'\nK-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
print('--------------------------------')
for fold in results:
    print(f'Fold {fold}: {results[fold]:.4f}')
print(f'Average: {average_accuracy:.4f}')

## ConvNeXt + PCA + SVM Classifier Training and Testing
As an extension to our SVM implementation, the paper suggests that applying PCA on the resulting features derives higher accuracy before being loaded into the SVM classifier. We implement this approach below, performing k-fold cross-validation testing on the PCA + SVM classifier, which is trained the on features extracted by our ConvNeXt model.

In [10]:
# Store the results of each fold
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=231)
results = {}

# Reduce dimensionality to 20 via PCA
n_components = 20

# Create the SVM classifier
pca_classifier = make_pipeline(
    StandardScaler(), 
    PCA(n_components=n_components), 
    SVC(kernel='linear')
)

# K-Fold Cross Validation
for fold, (train_indices, val_indices) in enumerate(kfold.split(train), 1):
    print(f'Fold {fold}')

    # Create data samplers for train and validation sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Create data loaders for train and validation sets
    train_loader = DataLoader(train, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(train, batch_size=32, sampler=val_sampler)
    
    # Extract features and labels for the training set
    train_features = []
    train_labels = []
    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            train_features.append(outputs.cpu().numpy())
            train_labels.append(labels.cpu().numpy())
    train_features = np.concatenate(train_features)
    train_labels = np.concatenate(train_labels)

    # Train the SVM classifier with PCA
    pca_classifier.fit(train_features, train_labels)
    
    # Extract features and labels for the validation set
    val_features = []
    val_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            val_features.append(outputs.cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    val_features = np.concatenate(val_features)
    val_labels = np.concatenate(val_labels)

    # Evaluate the classifier on the validation set
    val_predictions = pca_classifier.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)
    results[fold] = accuracy
    print(f'Fold {fold} Accuracy: {accuracy:.4f}')

# Print the average accuracy across all folds
average_accuracy = np.mean(list(results.values()))
print(f'\nK-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
print('--------------------------------')
for fold in results:
    print(f'Fold {fold}: {results[fold]:.4f}')
print(f'Average: {average_accuracy:.4f}')

Fold 1
Fold 1 Accuracy: 0.8953
Fold 2
Fold 2 Accuracy: 0.8807
Fold 3
Fold 3 Accuracy: 0.8860
Fold 4
Fold 4 Accuracy: 0.8747
Fold 5
Fold 5 Accuracy: 0.8933
K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 1: 0.8953
Fold 2: 0.8807
Fold 3: 0.8860
Fold 4: 0.8747
Fold 5: 0.8933
Average: 0.8860


### Testing and Metrics

Now with our trained models, we will now test with our test set and store metrics for each model. The metrics that we will store are the following:
- Accuracy
- Precision
- Recall
- F1

The metrics are defined in our paper more clearly, but to calculate these we will calculate the the following values:
- True Positive (TP)
- False Positive (FP)
- True Negative (TN)
- False Negative (FN)

We calculate these values below:

In [ ]:
# Extract features for training and validation sets
def extract_features(loader, model):
    features_list, labels_list = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            features = model(inputs)
            features = features.view(features.size(0), -1)
            features_list.append(features.cpu().numpy())
            labels_list.append(labels.cpu().numpy())
    return np.concatenate(features_list), np.concatenate(labels_list)

In [ ]:
# Extract Features
test_loader = DataLoader(test, batch_size=32, shuffle=False)
test_features, test_labels = extract_features(test_loader, model)

# List of trained classifiers
classifiers = {
    'SVM': svm_classifier,   # Assume svm_model is already trained
    'Softmax': softmax_classifier,  # Another trained classifier
    'SVM+PCA': pca_classifier   # Another trained classifier
}

# Dictionary to store results
results = {class_name: {} for class_name in classifiers}

# Evaluate each classifier
for class_name, classifier in classifiers.items():
    # Predict using the classifier
    test_predictions = classifier.predict(test_features)
    
    # Calculate metrics
    accuracy = accuracy_score(test_labels, test_predictions)
    precision = precision_score(test_labels, test_predictions, average='weighted')
    recall = recall_score(test_labels, test_predictions, average='weighted')
    f1 = f1_score(test_labels, test_predictions, average='weighted')
    
    # Store the results
    results[class_name]['accuracy'] = accuracy
    results[class_name]['precision'] = precision
    results[class_name]['recall'] = recall
    results[class_name]['f1'] = f1

    # Print the results
    print(f'{class_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

# Print a summary of the results
print('Comparison of Classifiers on Test Set:')
for clf_name in results:
    print(f'{clf_name}: Accuracy={results[clf_name]["accuracy"]:.4f}, Precision={results[clf_name]["precision"]:.4f}, Recall={results[clf_name]["recall"]:.4f}, F1 Score={results[clf_name]["f1"]:.4f}')